## RNN 创作古诗
在这一章中我们了解到循环神经网络非常擅长处理序列和自然语言处理，文本都是由单词或者汉字按照序列顺序组成的，那么如何能够生成文本呢？下面我们来讲一讲原理，需要你根据这个原理来实现整个网络。

### 原理介绍
前面我们介绍过 RNN 的输入和输出存在多种关系，比如多个输入对一个输出，这个时候输入是一个序列，输出是一个分类结果，就像使用 RNN 做图像分类。

这里我们使用 RNN 来生成文本，网络的输入是一个序列，同时输出也是一个相同长度的序列，结构如下

<img src=https://ws1.sinaimg.cn/large/006tNc79gy1fob5kq3r8jj30mt09dq2r.jpg width=700>

在上面的网络流程中，输入是一个序列 "床 前 明 月 光"，输出也是一个序列 "前 明 月 光 床"。如果你仔细观察可以发现网络的每一步输出都是下一步的输入，这就是其设计思路。

那么对于任意的一段话，比如 "我喜欢小猫"，我们可以将其拆分 "我 喜 欢 小 猫" 这个长度为 5 的序列，网络的每一步输出就是 "喜 欢 小 猫 我"，也就是每个字符的输出就是其**紧跟**的后一个字符。

当然对于一个序列，其最后一个字符后面并没有其他的字符，所以有多种方式选择，比如将序列的第一个字符作为其输出，也就是 "光" 的输出是 "床"，或者将其本身作为输出，也就是 "光" 的输出是 "光"，这里的选择可以有很多，我们使用一种循环的连接，将第一个字符作为最后一个字符的输出。

### 生成文本
这样设计网络的训练流程是为了非常好地生成文本，下面我们说明一下如何进行文本的生成。

首先需要输入网络一段初始的序列进行预热，预热的过程并不需要实际的输出结果，只是为了生成拥有记忆效果的隐藏状态，并将隐藏状态保留下来，接着我们开始正式生成文本，每个字符作为输入都可以得到输出，然后将输出作为下一步的输入，这样就可以不断地生成新的句子，这个过程是可以无限循环下去，或者到达我们的要求输出长度，具体可以看看下面的图示

<img src=https://ws2.sinaimg.cn/large/006tNc79gy1fob5z06w1uj30qh09m0sl.jpg width=800>

讲完了原理之后，下面就该你亲自动手去实现这个网络

In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf

首先我们可以探索一下数据集是什么样的

In [2]:
with open('./dataset/poetry.txt', 'r') as f:
    poetry_corpus = f.read()

我们取得了前100个字符的结果，其中 `\n` 表示换行符

In [3]:
poetry_corpus[:100]

'寒随穷律变，春逐鸟声开。\n初风飘带柳，晚雪间花梅。\n碧林青旧竹，绿沼翠新苔。\n芝田初雁去，绮树巧莺来。\n晚霞聊自怡，初晴弥可喜。\n日晃百花色，风动千林翠。\n池鱼跃不同，园鸟声还异。\n寄言博通者，知予物'

In [4]:
# 看看字符数
print('总的字符数: {}'.format(len(poetry_corpus)))

总的字符数: 942681


为了可视化比较方便，我们将换行字符 `\n` 替换成空格

In [5]:
poetry_corpus = poetry_corpus.replace('\n', ' ').replace('\r', ' ').replace('，', ' ').replace('。', ' ')
print(poetry_corpus[:100])

寒随穷律变 春逐鸟声开  初风飘带柳 晚雪间花梅  碧林青旧竹 绿沼翠新苔  芝田初雁去 绮树巧莺来  晚霞聊自怡 初晴弥可喜  日晃百花色 风动千林翠  池鱼跃不同 园鸟声还异  寄言博通者 知予物


### 文本数值表示
对于每个文字，电脑并不能像人一样能够有效地识别，所以必须做一个转换，将文字转换成电脑能够识别的数字，相当于每个不同的汉字，都用不同的数字去表示，可以对所有非重复的字符，从 0 开始建立索引

同时可能古诗中会出现一些生僻的字，这些字可能只会出现几次，甚至只会出现一次，引入这些字会增大模型的复杂度，同时也会影响模型的训练，可以将这些词频比较低的字去掉

关于汉字和数字的转换，我们已经为你实现好了一个转换器，感兴趣的同学可以去 `utils.py` 中查看，在之后的练习中，你可以使用这个转换器进行生成文本的转换，下面我们先看看例子

In [6]:
import numpy as np
from utils import TextConverter

In [7]:
convert = TextConverter('./dataset/poetry.txt', max_vocab=10000)


上面我们通过数据集建立好了这个转换器 `convert`，下面我们看看如何去调用

In [8]:
# 得到原始的文本结果
txt_char = poetry_corpus[:11]
print('原始的文本结果: {}'.format(txt_char))
print()

# 通过 convert 将文字转换成数字
num_char = convert.text_to_arr(txt_char)
print('转换成数字之后的结果: {}'.format(num_char))
print()

# 通过 convert 将数字转换成文字
origin_txt_char = convert.arr_to_text(num_char)
print('将数字重新转换成文字: {}'.format(origin_txt_char))

原始的文本结果: 寒随穷律变 春逐鸟声开

转换成数字之后的结果: [ 40 166 358 935 565   0  10 367 108  63  78]

将数字重新转换成文字: 寒随穷律变 春逐鸟声开


通过上面的例子，你可以看到，能够使用 `convert.text_to_arr` 对一个文本进行数字的转换，通过 `convert.arr_to_text` 将数字转换成文本 

### 构造时序样本数据
对于一整段文本，并不适合全部输入到循环神经网络中，因为我们前面了解到循环神经网络存在着长时依赖的问题，所以需要将整个文本分成很多个序列文本，然后将这些序列文本输入到循环神经网络中进行训练，只要我们定好每个序列的长度，那么序列个数也就被决定了。

In [9]:
# 每个序列的长度，你可以自行修改
n_step = 20

# 总的序列个数
num_seq = int(len(poetry_corpus) / n_step)

# 去掉最后不足一个序列长度的部分
text = poetry_corpus[:num_seq*n_step]

print('序列的个数: {}'.format(num_seq))

序列的个数: 47134


接着需要将序列中所有的文字转换成数字表示，同时重新排列成 **$(num\_seq \times n\_step)$** 的矩阵

完成下面的 `#todo` 的部分

In [10]:
arr = convert.text_to_arr(text) #todo: 使用 convert 将文本 text 转换成数字表示的数组
arr = arr.reshape(num_seq, n_step) #todo: 将转换之后的数组重新排列成 (num_seq x n_step) 的形状
arr = arr.astype(np.int32)

In [11]:
# 不要修改下面的代码
# ================== test =================
if arr.shape == (num_seq, n_step):
    print('Successful!')
else:
    print('Failed!')

Successful!


据此，我们可以构建 Tensorflow 中的数据读取来训练网络，这里我们将最后一个字符的输出 label 定为输入的第一个字符，也就是"床前明月光"的输出是"前明月光床"，完成下面 #todo 的部分

In [12]:
class TextDataset(object):
    def __init__(self, arr):
        self.arr = arr
        
    def __getitem__(self, item):
        #TODO: 取得 arr 中的 item 这一个序列
        x = self.arr[item, :]

        #TODO: 构造上述描述的 label
        y = np.zeros_like(x)
#         y = torch.zeros(x.size())
#         y = np.hstack((x[1:20], x[:1]))
        y[:-1], y[-1] = x[1:], x[0]

        return x, y
    
    def __len__(self):
        return self.arr.shape[0]

如果你构造好了这个数据集类，我们可以将其实例化

In [13]:
train_set = TextDataset(arr)

下面我们可以取出其中一个数据集参看一下是否是我们描述的这样，这个数据集需要像上面描述的一样，请自行检查

In [14]:
x, y = train_set[0]
# for x, y in train_set:
#     print('输入的文字序列 x: {}'.format(convert.arr_to_text(x)))
#     print('输出的文字序列 y: {}'.format(convert.arr_to_text(y)))
print("type(y):", type(y))
print('输入的文字序列 x: {}'.format(convert.arr_to_text(x)))
print('输出的文字序列 y: {}'.format(convert.arr_to_text(y)))

type(y): <class 'numpy.ndarray'>
输入的文字序列 x: 寒随穷律变 春逐鸟声开  初风飘带柳 晚
输出的文字序列 y: 随穷律变 春逐鸟声开  初风飘带柳 晚寒


### 建立模型
下面我们需要构建这个循环神经网路的网络结构，模型可以定义成非常简单的两层
- 第一层是 RNN 层, **LSTM (GRU)**
- 第二层是线性层，做分类问题，最后输出预测的字符 **slim.fully_connected**

只需要按照提示填写下面的 #todo 部分

- 构造输入

    首先构造一些placeholder作为网络的输入,方便之后代入数据, 需要构建的是:
    - inputs: placeholder, 接收 `[batch_size, n_step]` 的输入, 是输入的词
    - targets: placeholder, 接收 `[batch_size, n_step]` 的输入, 是输入词的对应词, 也就是 label
    - keep_prob: placeholder, 用来表示 dropout 的保留概率

In [15]:
def build_inputs(batch_size, n_step):
    '''
    
    args:
        batch_size: 一个批次中有多少个序列输入
        n_steps: 一个序列中有多少个词
        
    return:
        inputs: 输入的词
        targets: 输入词的对应词
        keep_prob: dropout 保留概率
    '''
    inputs = tf.placeholder(shape=(batch_size, n_step), dtype=tf.int32)
    targets = tf.placeholder(shape=(batch_size, n_step), dtype=tf.int32)
    keep_prob = tf.placeholder(name='keep_prob', dtype=tf.float32)
    
    return inputs, targets, keep_prob

- 构造 RNN

    然后我们开始构造 RNN, 将一个序列中的每个词产生一个输出词. 

    这里我们可以构造一个多层的 RNN, 可以使用 LSTM 或者 GRU 作为 RNN 的基本单元.

In [16]:
def build_rnn(hidden_size, num_layers, batch_size, keep_prob):
    '''
    
    args:
        keep_prob: dropout 保留概率
        hidden_size: RNN 隐藏层大小
        num_layers: RNN 隐藏层个数
        batch_size: batch size

    return:
        cell: RNN cell
        initial_state: RNN输入时的初始状态
    '''
    
    def build_cell(hidden_size, keep_prob):
        #todo: 得到一个 rnn cell, 可以是 rnn 或者 lstm 或者 gru
        lstm = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
        
        #todo: 添加 dropout
        lstm_drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return lstm_drop
    
    
    #todo: 得到一个多层的 rnn cell
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(hidden_size, keep_prob) for _ in range(num_layers)])
    
    #todo: 得到 cell 的初始状态
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

- 构造分类层

    现在我们要构造以 RNN 输出的结果为输入, 一个词为输出的全连接层, 这是一个分类问题, 有多少个词就是多少分类

In [17]:
import tensorflow.contrib.slim as slim

def build_output(rnn_out, in_size, out_size):
    '''
    
    args:
        rnn_out: 上一步rnn的输出
        in_size: rnn输出的特征个数
        out_size: 词的总个数(分类数)
    
    return:
        out: 输出词的概率向量
        logits: softmax之前的结果
    '''

    #todo: rnn_out 的形状是 (batch, n_step, rnn_size), 将形状改成 (batch x n_step, rnn_size)
    # 变成一个2阶矩阵才可以参与到下一步的分类层
#     seq_output = tf.concat(rnn_out, axis=1)
    x = tf.reshape(rnn_out, [-1, in_size])
    
    #todo: 一个全连阶层作为分类层
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
        
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    #todo: softmax 得到概率
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits,softmax_w,softmax_b

- 构造损失函数

    这是一个分类问题, 因此我们使用 softmax_with_logits 作为损失函数

In [18]:
def build_loss(logits, targets, lstm_size, num_classes, softmax_w, softmax_b):
    '''
    
    args:
        logits: softmax之前的结果
        targets: 目标词
        lstm_size: Number of LSTM hidden units
        num_classes: 词的总个数(分类数)
    
    return:
        loss: loss tensor.
    '''
    
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    #todo: softmax 分类损失函数
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_reshaped, logits=logits)
    loss = tf.reduce_mean(loss)
    
    return loss

- 构造训练过程

    接下来我们需要构造训练过程. 前面说到过, RNN 经常会遇到梯度爆炸的问题, 但有一种方法可以避免这类问题, 就是"梯度裁剪".
    
    tensorflow 可以通过**`tf.clip_by_global_norm(tensors, grad_clip)`**函数对`tensors`进行梯度裁剪. 
    
    形象地说, 如果大于`grad_clip`就会将大于的部分剪掉, 这样操作之后, 所有的`tensors`都比`grad_clip`要小, 也就不会存在爆炸的问题了.
    
    因此在这里我们会用到这个方法, 那么我们就不再使用最简单的`optimizer.minimize`这个函数去构造训练过程了, 需要把这个过程拆开, 具体来说分为下面几步:
    
        - 计算所有可训练变量的梯度
        - 对所有梯度进行裁剪
        - 再将梯度应用到原来的变量上去
        
    第一步和第二步应该都知道如何去做, 第三步需要用到一个全新的函数, **`optimizer.apply_gradients`**. 
    
    `optimizer`就是前面我们定义的比如说梯度下降法方法, Momentum方法, Adam方法等等优化器, 每个优化器都有`apply_gradients`方法, 这里不具体展开如何使用这个函数, 大家可以查看下面的函数说明或者参考[这里](https://tensorflow.google.cn/api_docs/python/tf/train/AdamOptimizer#apply_gradients)

---

<img src="https://image.ibb.co/dx7cRn/apply_gradient.png" alt="apply gradient" border="0" />

In [19]:
def build_optimizer(loss, learning_rate, grad_clip):
    '''
    
    args:
        loss: loss tensor
        learning_rate: 学习率
    
    return:
        optimizer: 优化方法
    '''

    #todo: 获取所有的可训练变量
    tvars = tf.trainable_variables()
    
    #todo: 获取 loss 对 tvars 的梯度
    grads = tf.gradients(loss, tvars)
    
    #todo: 使用 tf.clip_by_global_norm 进行梯度裁剪
    grads_clipped, _ = tf.clip_by_global_norm(grads, grad_clip)
#     grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), 5)
    
    #todo: 生成一个 Adam 优化器
    train_op = tf.train.AdamOptimizer(learning_rate)
    
    #todo: 使用 apply_gradients 生成一个参数更新 op
    optimizer = train_op.apply_gradients(zip(grads_clipped, tvars))
    
    return optimizer

- 构建完整的 **CharRNN**

In [20]:
class CharRNN:
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       rnn_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
        '''
        
        args:
            num_classes: 分类数, 也就是字符总数
            batch_size: batch size
            num_steps: 一个序列中出现的字符个数
            rnn_size: rnn 的隐藏层大小
            num_layers: rnn 中的隐藏层个数
            learning_rate: 学习率
            grad_clip: 梯度裁剪常数
            sampling: 是否进行采样
            
        '''
        # 之后我们用这个网络进行 inference 的时候, 我们会传入一个字符进来, 而不是训练时候的
        # 传入 n_step 个字符, 因此在这里用 sampling 来控制
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        with tf.name_scope('Inputs'):
        # Build the input placeholder tensors
            self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)
            
        #todo: 构建输入
        with tf.name_scope('Lstm'):
            cell, self.initial_state = build_rnn(rnn_size, num_layers, batch_size, self.keep_prob)
            
#         self.inputs, self.targets, self.keep_prob = tf.placeholder(tf.int32, [batch_size, None]),
#         tf.placeholder(tf.int32, [batch_size, None]),
#         tf.placeholder(tf.float32, name='keep_prob')

#         #todo: 构建RNN的cell
#         lstm = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size,state_is_tuple=True)     
#         #todo: 添加 dropout
#         lstm = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)   
#         #todo: 得到一个多层的 rnn cell
#         cell = tf.contrib.rnn.MultiRNNCell([lstm] * num_layers,state_is_tuple=True)
#         cell, self.initial_state = cell, cell.zero_state(batch_size, tf.float32)

        ### 用RNN跑一遍输入得到输出
        # 首先将输入转化成one_hot形式, 相当于给字符编码
        # 这里你也可以使用我们之前讲过的 word_embedding, 将字符嵌入到一个向量里面去
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        #todo: 运行RNN得到输出和最终状态(提示: 使用 tf.nn.dynamic_rnn)
        outputs, state = tf.nn.dynamic_rnn(cell,x_one_hot,initial_state=self.initial_state)
        
        #todo: 将最后的状态保存在 final_state 中
        self.final_state = state
        
        #todo: 得到分类层的结果
        self.prediction, self.logits, self.softmax_w,self.softmax_b = build_output(outputs, rnn_size, num_classes)
        
        #todo: 得到损失函数
        # Loss and optimizer (with gradient clipping)
        with tf.name_scope('Loss'):
            self.loss = build_loss(self.logits, self.targets, rnn_size, num_classes,self.softmax_w,self.softmax_b)
        
        #todo: 得到优化算子
        with tf.name_scope('SGD'):
            self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

接下来我们再定义网络的参数

In [21]:
# batch_size = None        # batch_size
# rnn_size = None          # rnn中隐藏层的大小
# num_layers = None        # rnn中隐藏层的个数
# learning_rate = None     # 学习率
# keep_prob = None         # dropout保留概率

batch_size = 256    # Sequences per batch
num_steps = 20    # Number of sequence steps per batch
rnn_size = 512     # Size of hidden layers in LSTMs
num_layers = 2         # Number of LSTM layers
learning_rate = 0.003    # Learning rate
keep_prob = 0.5     # Dropout keep probability

使用上面的`CharRNN`构造model

In [22]:
#todo
model =CharRNN(convert.vocab_size, batch_size=batch_size, num_steps=num_steps,
                rnn_size=rnn_size, num_layers=num_layers, 
                learning_rate=learning_rate)

In [23]:
epochs = 100

# Print losses every N interations
print_every_n = 1

# Save every N iterations
save_every_n = 1000

下面构造一个读取数据的`generator`, 也可以自行实现

In [24]:
import random

class build_data_generator:
    def __init__(self, data, batch_size, shuffle=False):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.nb_of_examples = len(data)
    
    def __call__(self):
        ind = 0
        indices = list(range(self.nb_of_examples))
        
        if self.shuffle:
            random.shuffle(indices)

        while ind + self.batch_size <= self.nb_of_examples:
            x, y = self.data[ind: ind + batch_size]
            ind += batch_size

            yield x, y

        return
    
    def __len__(self):
        return self.nb_of_examples // batch_size

In [25]:
import time

开始训练

In [26]:
saver = tf.train.Saver(max_to_keep=50)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    counter = 0
    
    for e in range(epochs):
        #todo: 
        #用 sess 去得到初始 state 保存在 new_state 中
        new_state = sess.run(model.initial_state)
        
        loss = 0
        dataset = build_data_generator(train_set, batch_size, True)
#         print("dataset:\n", dataset)
        
        for x, y in dataset():
            counter += 1
#             print("x:\n", x)
#             print("y:\n", y)
            start = time.time()
            
            #todo:
            # 构造 feed_dict
            # 这里, 我们需要得到model.inputs, model.targets, model.keep_prob, model.initial_state的输入
            # 需要将上一步得到的state作为这一步的model.initial_state
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            
            #todo:
            # 使用上面定义的 feed_dict, 运行 session, 获得当前 batch 的 loss, state, 并运行 model.optimizer
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/checkpoints/i{}_l{}.ckpt".format(counter, rnn_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, rnn_size, batch_loss))

Epoch: 1/100...  Training Step: 1...  Training loss: 8.5918...  1.7820 sec/batch
Epoch: 1/100...  Training Step: 2...  Training loss: 8.4878...  0.3435 sec/batch
Epoch: 1/100...  Training Step: 3...  Training loss: 9.4271...  0.3306 sec/batch
Epoch: 1/100...  Training Step: 4...  Training loss: 7.4339...  0.3334 sec/batch
Epoch: 1/100...  Training Step: 5...  Training loss: 7.6500...  0.3351 sec/batch
Epoch: 1/100...  Training Step: 6...  Training loss: 6.9588...  0.3350 sec/batch
Epoch: 1/100...  Training Step: 7...  Training loss: 6.6767...  0.3340 sec/batch
Epoch: 1/100...  Training Step: 8...  Training loss: 6.5810...  0.3349 sec/batch
Epoch: 1/100...  Training Step: 9...  Training loss: 6.6424...  0.3346 sec/batch
Epoch: 1/100...  Training Step: 10...  Training loss: 6.6177...  0.3337 sec/batch
Epoch: 1/100...  Training Step: 11...  Training loss: 6.3776...  0.3323 sec/batch
Epoch: 1/100...  Training Step: 12...  Training loss: 6.1868...  0.3315 sec/batch
Epoch: 1/100...  Training

Epoch: 1/100...  Training Step: 102...  Training loss: 5.6661...  0.3365 sec/batch
Epoch: 1/100...  Training Step: 103...  Training loss: 5.8227...  0.3362 sec/batch
Epoch: 1/100...  Training Step: 104...  Training loss: 5.7235...  0.3370 sec/batch
Epoch: 1/100...  Training Step: 105...  Training loss: 5.7773...  0.3363 sec/batch
Epoch: 1/100...  Training Step: 106...  Training loss: 5.7724...  0.3365 sec/batch
Epoch: 1/100...  Training Step: 107...  Training loss: 5.7560...  0.3349 sec/batch
Epoch: 1/100...  Training Step: 108...  Training loss: 5.5436...  0.3353 sec/batch
Epoch: 1/100...  Training Step: 109...  Training loss: 5.5355...  0.3327 sec/batch
Epoch: 1/100...  Training Step: 110...  Training loss: 5.5502...  0.3359 sec/batch
Epoch: 1/100...  Training Step: 111...  Training loss: 5.5329...  0.3383 sec/batch
Epoch: 1/100...  Training Step: 112...  Training loss: 5.6993...  0.3343 sec/batch
Epoch: 1/100...  Training Step: 113...  Training loss: 5.6548...  0.3372 sec/batch
Epoc

Epoch: 2/100...  Training Step: 201...  Training loss: 5.7605...  0.3374 sec/batch
Epoch: 2/100...  Training Step: 202...  Training loss: 5.5873...  0.3400 sec/batch
Epoch: 2/100...  Training Step: 203...  Training loss: 5.6592...  0.3404 sec/batch
Epoch: 2/100...  Training Step: 204...  Training loss: 5.6164...  0.3391 sec/batch
Epoch: 2/100...  Training Step: 205...  Training loss: 5.6804...  0.3373 sec/batch
Epoch: 2/100...  Training Step: 206...  Training loss: 5.6030...  0.3350 sec/batch
Epoch: 2/100...  Training Step: 207...  Training loss: 5.6373...  0.3371 sec/batch
Epoch: 2/100...  Training Step: 208...  Training loss: 5.6952...  0.3343 sec/batch
Epoch: 2/100...  Training Step: 209...  Training loss: 5.6030...  0.3355 sec/batch
Epoch: 2/100...  Training Step: 210...  Training loss: 5.5529...  0.3355 sec/batch
Epoch: 2/100...  Training Step: 211...  Training loss: 5.5456...  0.3359 sec/batch
Epoch: 2/100...  Training Step: 212...  Training loss: 5.5536...  0.3368 sec/batch
Epoc

Epoch: 2/100...  Training Step: 300...  Training loss: 5.3303...  0.3370 sec/batch
Epoch: 2/100...  Training Step: 301...  Training loss: 5.5105...  0.3354 sec/batch
Epoch: 2/100...  Training Step: 302...  Training loss: 5.3467...  0.3362 sec/batch
Epoch: 2/100...  Training Step: 303...  Training loss: 5.3109...  0.3369 sec/batch
Epoch: 2/100...  Training Step: 304...  Training loss: 5.4235...  0.3374 sec/batch
Epoch: 2/100...  Training Step: 305...  Training loss: 5.6227...  0.3385 sec/batch
Epoch: 2/100...  Training Step: 306...  Training loss: 5.5127...  0.3352 sec/batch
Epoch: 2/100...  Training Step: 307...  Training loss: 5.4087...  0.3357 sec/batch
Epoch: 2/100...  Training Step: 308...  Training loss: 5.3785...  0.3385 sec/batch
Epoch: 2/100...  Training Step: 309...  Training loss: 5.3132...  0.3355 sec/batch
Epoch: 2/100...  Training Step: 310...  Training loss: 5.3090...  0.3395 sec/batch
Epoch: 2/100...  Training Step: 311...  Training loss: 5.3995...  0.3363 sec/batch
Epoc

Epoch: 3/100...  Training Step: 399...  Training loss: 5.3120...  0.3376 sec/batch
Epoch: 3/100...  Training Step: 400...  Training loss: 5.3051...  0.3401 sec/batch
Epoch: 3/100...  Training Step: 401...  Training loss: 5.0679...  0.3364 sec/batch
Epoch: 3/100...  Training Step: 402...  Training loss: 5.1430...  0.3377 sec/batch
Epoch: 3/100...  Training Step: 403...  Training loss: 5.2891...  0.3377 sec/batch
Epoch: 3/100...  Training Step: 404...  Training loss: 5.4554...  0.3361 sec/batch
Epoch: 3/100...  Training Step: 405...  Training loss: 5.3807...  0.3377 sec/batch
Epoch: 3/100...  Training Step: 406...  Training loss: 5.4038...  0.3359 sec/batch
Epoch: 3/100...  Training Step: 407...  Training loss: 5.2884...  0.3366 sec/batch
Epoch: 3/100...  Training Step: 408...  Training loss: 5.2730...  0.3385 sec/batch
Epoch: 3/100...  Training Step: 409...  Training loss: 5.3838...  0.3368 sec/batch
Epoch: 3/100...  Training Step: 410...  Training loss: 5.3357...  0.3399 sec/batch
Epoc

...

Epoch: 98/100...  Training Step: 17989...  Training loss: 3.7370...  0.3434 sec/batch
Epoch: 98/100...  Training Step: 17990...  Training loss: 3.7761...  0.3428 sec/batch
Epoch: 98/100...  Training Step: 17991...  Training loss: 3.7144...  0.3416 sec/batch
Epoch: 98/100...  Training Step: 17992...  Training loss: 3.6896...  0.3393 sec/batch
Epoch: 98/100...  Training Step: 17993...  Training loss: 3.7509...  0.3424 sec/batch
Epoch: 98/100...  Training Step: 17994...  Training loss: 3.7124...  0.3398 sec/batch
Epoch: 98/100...  Training Step: 17995...  Training loss: 3.7414...  0.3433 sec/batch
Epoch: 98/100...  Training Step: 17996...  Training loss: 3.7591...  0.3403 sec/batch
Epoch: 98/100...  Training Step: 17997...  Training loss: 3.7742...  0.3413 sec/batch
Epoch: 98/100...  Training Step: 17998...  Training loss: 3.6792...  0.3406 sec/batch
Epoch: 98/100...  Training Step: 17999...  Training loss: 3.7239...  0.3407 sec/batch
Epoch: 98/100...  Training Step: 18000...  Training lo

Epoch: 99/100...  Training Step: 18085...  Training loss: 3.8065...  0.3414 sec/batch
Epoch: 99/100...  Training Step: 18086...  Training loss: 3.7839...  0.3407 sec/batch
Epoch: 99/100...  Training Step: 18087...  Training loss: 3.7979...  0.3452 sec/batch
Epoch: 99/100...  Training Step: 18088...  Training loss: 3.8141...  0.3449 sec/batch
Epoch: 99/100...  Training Step: 18089...  Training loss: 3.7589...  0.3431 sec/batch
Epoch: 99/100...  Training Step: 18090...  Training loss: 3.7936...  0.3456 sec/batch
Epoch: 99/100...  Training Step: 18091...  Training loss: 3.7061...  0.3449 sec/batch
Epoch: 99/100...  Training Step: 18092...  Training loss: 3.6718...  0.3412 sec/batch
Epoch: 99/100...  Training Step: 18093...  Training loss: 3.7408...  0.3415 sec/batch
Epoch: 99/100...  Training Step: 18094...  Training loss: 3.7496...  0.3431 sec/batch
Epoch: 99/100...  Training Step: 18095...  Training loss: 3.7367...  0.3459 sec/batch
Epoch: 99/100...  Training Step: 18096...  Training lo

Epoch: 99/100...  Training Step: 18181...  Training loss: 3.7486...  0.3426 sec/batch
Epoch: 99/100...  Training Step: 18182...  Training loss: 3.6699...  0.3440 sec/batch
Epoch: 99/100...  Training Step: 18183...  Training loss: 3.7264...  0.3464 sec/batch
Epoch: 99/100...  Training Step: 18184...  Training loss: 3.7236...  0.3404 sec/batch
Epoch: 99/100...  Training Step: 18185...  Training loss: 3.7107...  0.3450 sec/batch
Epoch: 99/100...  Training Step: 18186...  Training loss: 3.7634...  0.3408 sec/batch
Epoch: 99/100...  Training Step: 18187...  Training loss: 3.7602...  0.3416 sec/batch
Epoch: 99/100...  Training Step: 18188...  Training loss: 3.7945...  0.3422 sec/batch
Epoch: 99/100...  Training Step: 18189...  Training loss: 3.7239...  0.3428 sec/batch
Epoch: 99/100...  Training Step: 18190...  Training loss: 3.7957...  0.3417 sec/batch
Epoch: 99/100...  Training Step: 18191...  Training loss: 3.6429...  0.3413 sec/batch
Epoch: 99/100...  Training Step: 18192...  Training lo

Epoch: 100/100...  Training Step: 18276...  Training loss: 3.6642...  0.3417 sec/batch
Epoch: 100/100...  Training Step: 18277...  Training loss: 3.7261...  0.3444 sec/batch
Epoch: 100/100...  Training Step: 18278...  Training loss: 3.7540...  0.3428 sec/batch
Epoch: 100/100...  Training Step: 18279...  Training loss: 3.7448...  0.3388 sec/batch
Epoch: 100/100...  Training Step: 18280...  Training loss: 3.7821...  0.3402 sec/batch
Epoch: 100/100...  Training Step: 18281...  Training loss: 3.7097...  0.3436 sec/batch
Epoch: 100/100...  Training Step: 18282...  Training loss: 3.7069...  0.3445 sec/batch
Epoch: 100/100...  Training Step: 18283...  Training loss: 3.6887...  0.3404 sec/batch
Epoch: 100/100...  Training Step: 18284...  Training loss: 3.6678...  0.3409 sec/batch
Epoch: 100/100...  Training Step: 18285...  Training loss: 3.7282...  0.3417 sec/batch
Epoch: 100/100...  Training Step: 18286...  Training loss: 3.7640...  0.3389 sec/batch
Epoch: 100/100...  Training Step: 18287... 

Epoch: 100/100...  Training Step: 18371...  Training loss: 3.7679...  0.3412 sec/batch
Epoch: 100/100...  Training Step: 18372...  Training loss: 3.8026...  0.3447 sec/batch
Epoch: 100/100...  Training Step: 18373...  Training loss: 3.7478...  0.3416 sec/batch
Epoch: 100/100...  Training Step: 18374...  Training loss: 3.7725...  0.3403 sec/batch
Epoch: 100/100...  Training Step: 18375...  Training loss: 3.6509...  0.3402 sec/batch
Epoch: 100/100...  Training Step: 18376...  Training loss: 3.7214...  0.3400 sec/batch
Epoch: 100/100...  Training Step: 18377...  Training loss: 3.7156...  0.3400 sec/batch
Epoch: 100/100...  Training Step: 18378...  Training loss: 3.7085...  0.3438 sec/batch
Epoch: 100/100...  Training Step: 18379...  Training loss: 3.6613...  0.3434 sec/batch
Epoch: 100/100...  Training Step: 18380...  Training loss: 3.6372...  0.3448 sec/batch
Epoch: 100/100...  Training Step: 18381...  Training loss: 3.6983...  0.3447 sec/batch
Epoch: 100/100...  Training Step: 18382... 

然后我们可以查看在 checkpoints 中所有保存的模型

In [27]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i18400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i4000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i5000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i6000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i7000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i8000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i9000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i10000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i11000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i12000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/checkpoints/i13000_l512.ckpt"
all_model_checkpoint_paths: "che

在训练完成之后, 可以从 checkpoints 中恢复模型, 然后我们再给网络输入一个字符, 再让 CharRNN 不断生成新的字符, 也就是让神经网络"写诗"

为了让输出的字符更加丰富随意, 这里我们从模型输出的概率向量中随机选取前几个中的一个作为最后的输出

In [28]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

现在是时候看看我们训练的神经网络能够写出什么样的诗了, 这里封装成了一个 sample 函数, 需要完成下面的 #todo 部分

注意, 我们在用 CharRNN 写诗的时候, 需要先用输入的字符对网络进行"预热", 这个过程我们不采用网络输出字符的结果, 但是可以得到一个更好的状态, 然后就可以用这个状态作为后续生成新字符的初始状态, 从而获得更好的效果

In [29]:
def sample(checkpoint, n_samples, rnn_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    
    # 构造CharRNN模型
    # 注意, 这里我们处于测试状态, batch_size 和 n_step 都应为1
    model = CharRNN(convert.vocab_size, rnn_size=rnn_size, sampling=True)
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)

        #todo:
        # 得到初始 state
        new_state = sess.run(model.initial_state)
        
        # 这一步我们先将输入的几个字符对网络进行"预热", 
        # 这样可以得到更好的 state
        for c in prime:
            x = np.zeros((1, 1))
            x[0, 0] = convert.word_to_int(c)
            
            #todo:
            # 像之前一样设定feed_dict
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            
            #todo:
            # 得到概率输出以及当前输出状态
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, convert.vocab_size)
        samples.append(convert.int_to_word(c))

        for i in range(n_samples):
            x[0,0] = c
            
            #todo:
            # 像之前一样设定feed_dict
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            
            #todo:
            # 得到概率输出以及当前输出状态
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, convert.vocab_size)
            samples.append(convert.int_to_word(c))
        
    return ''.join(samples)

看一下训练时长最长的模型的效果

In [30]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, convert.vocab_size, rnn_size, convert.vocab_size, prime="天青色等烟雨")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i18400_l512.ckpt
天青色等烟雨无看年夜  在  何人有路心  不为学天人  故国上云人  凭山宋叶衣  危葭落夕云  庭河上望云  吾思寄何游  话  长人入上声  谁看上塞人  空看楚塞秋月风  莫叹如兹时  知君亦为襟  大禽飞水下  兹尺学金饰  素  天山在为心  萧洲逼蜀时  花山动白楼  春风落落风  江山上雾新霭风  如知圣柄明  当朝帝帝骢  明时意时行  何夜上云尘  愿言命上来  故门多此人  寒门满暮楼  更来天月深  吾君载古心  自知千涧云   翩盖降车列  共受丰中年  金旆在文台  徒此圣花开  圣光方大回  金飞织管空        风  秋  春风一不心  一怜支署微  还来思落山  山泉生竹生  寒吟向夏门  孤风隔海桥  无看入雪山  何堪白里中  谢君策水深  萧桡思钓然  登余入海庭  江流隔汉城  几朝杨桂尘  裴条有此情  空看映蕙文  佳传奉彩文  六媒四表书  清飘玉帐舒  含门意应知  何能素妾情  妾声看芳叶  相对花光清月同  妾妾折罗台月    光妾娇  上袖纤妆叶  拂舞含光扇  虚乐表秦恩  仙  天 何日上中开  清首拂花声  新门绝火风  无逢老此风  君言黄士来  因朝杨月来  关林月月寒  还因比化衣  金驾大川城上人  东山在水楼  春风雁路深  江峰洒洞衣  楚人过顶稀  茅冰落晚寒山心  一念在中情  相言白道时  青袍入石篱  终来得古年  眼冰桃玉冰空     风山映尘发  王条意幽曲月香  红粉拂飞衣  乍彩湿花飞  无  何去是无人  一窥鹓塞尘  平山不在时为风  早  明日复如里  自言尘贵玉不风  春看发花人  一为赴塞人  烟河落树斜  孤馆信无行  天枫北里来  行来发水楼  春风吹不行  一从出上中  谁知车下春    □风逐天发  因当二宰人  生精礼圣知  朝天渭水来  凭将逐镜台  花看映翠风  风风散彩罗  六士敌鱼时  吾君重恶簪  圣然游钓邑人归  不著入荆枝  结听黄风襟  周符神礼馀  天晨意中新  香  何风不如归  清看缑殿衣  江山有望飞山山  一是雪花时  莫是见君人  江看西远城  东城游雁归  故日戎铜衣人   风得映兰萼

看一下刚刚开始训练时模型的效果

In [32]:
checkpoint = 'checkpoints/checkpoints/i1000_l512.ckpt'
samp = sample(checkpoint, convert.vocab_size, rnn_size, convert.vocab_size, prime="天青色等烟雨")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/checkpoints/i1000_l512.ckpt
天青色等烟雨人山人  何是自中中  不山不上时  今风自自春  何风入自人  今  山人一自风       人山入自人      风风入人时  今人一路风                   风风有自时                山年有自人  不风不不风  何山一不春  不风入上中    何风一相人  山日不人归  不山不上人  风风见不时                        风人入不中  今  山人入相时  何日一人中    不年入自中                        春山不不风        山人入相中  何日不相春  一风有不春  山风一不人  何日不山中    何年一山人  何  不人入自人  山日有相人  何水入山归  山  山  春  何风有不春                     风山不自中                                                  风山入路人  今人有上中  今人不路人  今人不上中  何风不不人  山山有自人  何风不不风  今山有水人  不人不水春    何日不山中          春人不上人  何日自相时  何  山风入不稀  不风不不风  一山无水生  今人不上风                       山人不不时  何风有自春  风人不自人  何人一自中  不  山风入人时  山人见自稀  何  何人不水人  风  何人一路归                                  风年不不风  不年入上人  不人入路中      何人一不时       风年入上春    风风有不春无山上  不年一山中  不人不不人                                 山山不不人  何山不路时       人山在上人                                风山有路年  不山有水中  风年入上人       山生一不时  今风入水中                                                         

In [33]:
checkpoint = 'checkpoints/checkpoints/i5000_l512.ckpt'
samp = sample(checkpoint, convert.vocab_size, rnn_size, convert.vocab_size, prime="天青色等烟雨")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/checkpoints/i5000_l512.ckpt
天青色等烟雨  花歌玉月开  金山玉日筵    不中不天水  天山不自中  天人何日归  风云出不人  何山日月人  何人日上人  山山入不人  何人日水中  长来日日衣  何来日不中  长风出不来  何人出上归  云来不上心  无山无日心  无风日水归  风山日月归不不里      风山日风    青人风    山人不水衣  天来不不中  何山日水人  风云日上中  山来不上归  山风不不中  风人有水归  风风不未深  谁君何上心  不君无不来  长人望上来  山来有月人  空来入月人  长山入上来  云山有上中  无山日日来  山山日日归  空云入上流  长风不未中  山山无里人  风来不月归  风云有不中  何人不月心  还风不未人  不人一月来  长云日不中  何风日上来  山人一上中  长人入水来  山云入不中  山来入上人  长风一水归何中去  月  风风一云树  山山何不人  何人有水归  山山不水中  何风入水中  空山不不中  长山入不时  何人日不人  何来出月人  风云日不来  山人望上归  山山不水来  云云玉上来  山山入不深  何山日水人  长人有日来  何来入月来  空云日日归  山人不日深  山山何里心  何山日上深  无人无月心  空山落月流  风山日月人  山来入水深  长人何月来  何来不未深  江风一未来  江风不上心  相山日不深  山人有月归  空山日上来  风人日日中  长来不上深  山山无上来  何来落水来  山来不未归  长人入不人  江人不不人  不风有水来  山来不上归  山山落月归  风风不上深  长人不上来  风山出水人  云风日路人  山云落上开  江人不日深  山人一水深  长山有里人  江山不不深  山来不不归  长来不不归  何年日不春  高山有不时  不山有水来  山山入月归  山风不不春  风人不不春  风风不水人  风人日月来  长人入月人  空云不日归  山人不不人  山来不未人  何来何月来  何人不北人  何来出不人  风人不不春  风来不水人  风风日未中  山风日上来  云风有月人  云来不上春  风来入不时  何来不不来

看看模型的中间结果

In [34]:
checkpoint = 'checkpoints/checkpoints/i10000_l512.ckpt'
samp = sample(checkpoint, convert.vocab_size, rnn_size, convert.vocab_size, prime="天青色等烟雨")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/checkpoints/i10000_l512.ckpt
天青色等烟雨  含花意月流  青山日素生龙归  不与山山心  云日见花深  江色宿人间    弱马白云存  一事有人衣  白日出山山  野发见山人  野  何此风已流  不为无此归  春云见路深  白山山水间山云  不在不沾城  山人日云路人人  独里夜云人  不雪暮风人  相知白上深  山门秋水心不人  不风风水归  江云发路稀  孤风日水深  月山山路间  山山入木中    风  何日不云山  江城雨雨深  山山见不人  江山过客深  高山独暮山  江风不客归有花暮人山  不月有关归  山日意何人  山花月叶清  寒声入客人  故人思路稀  春云日雨归  江山草水稀  江花草雪云有花路人长  江事有山秋  云雨入云流  相君无不人    风君此不云  寒山日木中  江花云雪长长山  一日风山山  不与不门城  月月过知过人人  山月夜人城  云月落花水未人  风风积竹山    鸟  不来不未见  山波不不时  孤风不自人    云风何未时  风山花水花  野家山故生  何思有山归无山雨人人  一月有人深  江思人不春  江风落月中    马  白日有山见  归思思不时  野风吹白移  不知云远间  春风月水间  山帆水木人  江山见一年  风河白木深  江风积竹流    水  不君思不时  离风此见时  山风风海人  江来日路人  江城日海山  山云白路深  白风春上山  云风入草深  江山天水来   山上上人人  山风水海云  江人见帝臣  相时白路时  风花落照声  莫君秋草间  东山秋草间  东江秋自人  山花出水流    风山白不死  孤公就白行人归  云草出山城  野日过山下自归  寒山入色来   风见有无时  幽上独东山  风望一风深  何有整门中  素烛带新阴自山  山为白沈间  一月白山时  高风积水低   山雪竹新衣  山风竹木流  山山水照新山云月  不暮空不发  自日惜麟心  野  山云见不春  山光落水中  江山水水深  白心不不归有人  风花白紫香  高条芳节稀  山花带月时  江云白雨阴  禅门不水人  月人寒上来云山  自人秋路时  江山过雪船  风云雪上山  有来寒上

可以看到, 随着模型不断训练, 得到的语句越来越丰富, 越来越完整, 也就是说效果越来越好. 但是本质上这还是一个概率模型, 